<a href="https://colab.research.google.com/github/sibyl-dev/Explingo/blob/main/basic_llm_setup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openai

In [ ]:
import openai
from google.colab import userdata

In [ ]:
# Perform tasks using OpenAI API
client = openai.OpenAI(
  api_key=userdata.get('openai'),
)

In [ ]:
from openai import OpenAI

prompt =  ("You are a helpful assistant. "
           "You are talking to a user who is not familiar with ML or data science. "
           "You are helping them understand their ML model's predictions.")
question =  ("Convert this feature contibution explanation, generated using SHAP, into a simple narrative. "
             "The explanation is presented in (feature, feature_value, contribution) format: ")
explanation = ("")

response = client.chat.completions.create(
  model="gpt-3.5-turbo-0125",
  messages=[
    {"role": "system", "content": prompt},
    {"role": "user", "content": question + explanation}
  ],
  max_tokens=32
)

In [ ]:
for choice in response.choices:
  print(choice.message.content)